 <center> <h1> <b> Pattern Recognition and Machine Learning (EE5610 - EE2802 - AI2000 - AI5000) </b> </h1> </center>

<b> Programming Assignment - 03 : Classification</b>


This programming assignment offers you an opportunity to implement various linear classification models. You'll begin with straightforward algorithms like Least Squares classification, LDA, and FDA. Subsequently, you'll delve into implementing the perceptron algorithm. Lastly, you'll tackle probabilistic approaches for classification. To deepen your understanding of these methods, you'll compare their performance and application scenarios critically.

<b> Instructions </b>
1. Plagiarism is strictly prohibited.
2. Delayed submissions will be penalized with a scaling factor of 0.5 per day.
3. Please DO NOT use any machine learning libraries unless and otherwise specified.








**Part-1) Least squares approach to classification**

a). Data generation: Consider a classification scenario with two classes. Class 1 follows a Gaussian distribution with a mean vector $\begin{bmatrix} 1  \\ 1  \end{bmatrix}$ and a covariance matrix $\begin{bmatrix} 0.3 & 0.0  \\ 0.0 & 0.3 \end{bmatrix}$. Class 2 follows a Gaussian distribution with a mean vector $\begin{bmatrix} 2  \\ 2  \end{bmatrix}$ and a covariance matrix $\begin{bmatrix} 0.3 & 0.0  \\ 0.0 & 0.3 \end{bmatrix}$. Employ these class distributions to generate 50 samples per class for training and 200 samples for testing.

b). Find the decision boundary: Assign the target value "0" to all data points in class 1 and the target value "1" to all data points in class 2. Implement the pseudo-inverse solution to determine the weight vector, which represents the decision boundary.


c). Evaluate the quality of decision boundary: A decision boundary is deemed effective if it correctly classifies all data points. Accuracy measures the performance of a classification problem. Calculate the accuracy of the test data using the decision boundary obtained from the pseudo-inverse solution.


d). Visualize the decision boundary: You can either display the decision hyperplane or use different colors to distinguish regions corresponding to the two classes on a 2D plane. Superimpose the test points onto the decision boundary plot.

e). Assign the value "-1" to class1 and the value "+1" to class 2. Repeat the experiments.

f). Experiment with Laplacian distribution and compare the performance of it with the Gaussian distribution experiments

g. Report your observations.




In [ ]:
#All imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import random
import math

In [ ]:
########################################
#Part-1)-a: Data generation
########################################

#Set the parameters of class distribution
mean1 = [1,1]           #Mean of class1
mean2 = [2,2]           #Mean of class2
cov = [[0.3,0],[0,0.3]] #Covariance matrix. Same covariance for class1 and class2

#Generate training data. You can use "np.random.multivariate_normal()" function to sample data points from multivariate Gaussian distribution
class1_tr = np.random.multivariate_normal(mean1, cov, 50)            #Class1 training data
class2_tr = np.random.multivariate_normal(mean2, cov, 50)            #Class2 training data
tr_data = np.concatenate((class1_tr, class2_tr))                 #Combine class1 training and class2 training data to get total training data
tr_targets = np.array([0 if i < len(tr_data) / 2 else 1 for i in range(len(tr_data))])           #Class labels. Assign the value "0" to class1 and the value "1" to class2.

# #Generate testing data.
class1_te = np.random.multivariate_normal(mean1, cov, 200)           #Class1 testing data
class2_te = np.random.multivariate_normal(mean2, cov, 200)            #Class2 testing data
te_data = np.concatenate((class1_te,class2_te))              #Combine class1 testing and class2 testing data to get total testing data
te_targets = np.array([0 if i < len(te_data) / 2 else 1 for i in range(len(te_data))])           #Class labels. Assign the value "0" to class1 and the value "1" to class2.

threshold = 0.5
########################################
#Part-1)-b: Find the decision boundary
########################################
#Complete the below function
def LS_Classify(X_train, Y_train, X_test):
    #Inputs: Training data, Training labels, and Testing data
    #Outputs: Testing labels

    #Impliment pseudo inverse solution to get the weight vector
    X = []
    T = []
    for x in X_train:
      X.append(np.array([1, x[0],x[1]]))

    X = np.array(X)
    for i in Y_train:
      if i != 1:
        T.append(np.array([1,0]))
      else:
        T.append(np.array([0,1]))

    T = np.array(T)

    w = np.linalg.inv(X.T @ X) @ X.T @ T

    Y = []
    for y in X_test:
      Y.append(np.array([1, y[0], y[1]]))

    Y = np.array(Y)
    #Predict the lables of test data using the pseudo inverse solution
    targets = Y @ w
    predictions = []
    for t in targets:
      if t[0] > threshold:
        predictions.append(0)
      else:
        predictions.append(1)

    predictions = np.array(predictions)

    #Return the predicted test labels
    return w, predictions

#Get the test data predictions
W, predictions = LS_Classify(tr_data, tr_targets, te_data)

##########################################
#Part-1)-c: Evaluate the quality of decision boundary
##########################################
#Complete the below function
def LS_Classify_Accuracy(Y_test, Y_pred):
    #Inputs: Ground truth test labels and predicted test labels
    #Outputs: Accuracy

    #Compute the accuracy
    count = 0
    for i, p in enumerate(Y_pred):
      if p == Y_test[i]:
        count += 1

    accuracy = count / len(Y_test)
    #Return the accuracy
    return accuracy*100

print("Accuracy: ",LS_Classify_Accuracy(te_targets, predictions))
##########################################
#Part-1)-d: Visualize the decision boundary
##########################################
def decision_boundary(W,x):
  y = []
  w2 = np.array([W[0][1], W[1][1], W[2][1]])
  w1 = np.array([W[0][0], W[1][0], W[2][0]])
  w = w1 - w2
  w /= np.linalg.norm(w)
  for e in x:
    y.append(-(w[1] / w[2])*e - (w[0] / w[2]))

  return y

class1_pred = []
class2_pred = []
for i, p in enumerate(predictions):
  if p != 1:
    class1_pred.append(te_data[i])
  else:
    class2_pred.append(te_data[i])

class1_pred = np.array(class1_pred)
class2_pred = np.array(class2_pred)
class1_te = np.array(class1_te)
class2_te = np.array(class2_te)

x = np.linspace(min(np.min(class1_te[:,0]), np.min(class2_te[:,0])) - 1, max(np.max(class1_te[:,0]), np.max(class2_te[:,0])) + 1, 1000)
y = decision_boundary(W, x)
fig, axes = plt.subplots(1, 2, figsize = (15,6))
fig.suptitle("Gaussian Distribution")
axes[0].scatter(class1_te[:,0],class1_te[:, 1], label = "Class 1")
axes[0].scatter(class2_te[:,0],class2_te[:, 1], label = "Class 2")
axes[0].set_title("Original Classification of Points")
axes[0].grid(True)
axes[0].legend()

axes[1].scatter(class1_pred[:,0],class1_pred[:, 1], label = "Class 1 predicted")
axes[1].scatter(class2_pred[:,0],class2_pred[:, 1], label = "Class 2 predicted")
axes[1].plot(x, y, label = "Decision Boundary")
axes[1].set_title("Predicted Classification of Points")
axes[1].grid(True)
axes[1].legend()

plt.show()

##########################################
#Part-1)-e: Change the target label notation, and repeat the experiments
##########################################
new_tr_targets = np.array([-1 if i < len(tr_data) / 2 else 1 for i in range(len(tr_data))])  # Assume -1 for class1 and +1 for class 2
new_te_targets = np.array([-1 if i < len(te_data) / 2 else 1 for i in range(len(te_data))])

W, predictions = LS_Classify(tr_data, new_tr_targets, te_data)
for i in range(len(predictions)):
  if predictions[i] == 0:
    predictions[i] = -1

print("New Accuracy: ",LS_Classify_Accuracy(new_te_targets, predictions))

class1_pred = []
class2_pred = []
for i, p in enumerate(predictions):
  if p != 1:
    class1_pred.append(te_data[i])
  else:
    class2_pred.append(te_data[i])

class1_pred = np.array(class1_pred)
class2_pred = np.array(class2_pred)
class1_te = np.array(class1_te)
class2_te = np.array(class2_te)

y = decision_boundary(W, x)
fig, axes = plt.subplots(1, 2, figsize = (15,6))
fig.suptitle("Plots with updated targets(Gaussian Distribution)")
axes[0].scatter(class1_te[:,0],class1_te[:, 1], label = "Class 1")
axes[0].scatter(class2_te[:,0],class2_te[:, 1], label = "Class 2")
axes[0].set_title("Original Classification of Points")
axes[0].grid(True)
axes[0].legend()

axes[1].scatter(class1_pred[:,0],class1_pred[:, 1], label = "Class 1 predicted")
axes[1].scatter(class2_pred[:,0],class2_pred[:, 1], label = "Class 2 predicted")
axes[1].plot(x, y, label = "Decision Boundary")
axes[1].set_title("Predicted Classification of Points")
axes[1].grid(True)
axes[1].legend()

plt.show()

##########################################
#Part-1)-f: Repeat the experiments with Laplacial distribution
##########################################
determinant = np.linalg.det(cov)

#Generate training data. You can use "np.random.multivariate_normal()" function to sample data points from multivariate Gaussian distribution
class1_tr = np.random.laplace(mean1, cov[0][0], (50,2))            #Class1 training data
class2_tr = np.random.laplace(mean2, cov[1][1], (50,2))            #Class2 training data
tr_data = np.concatenate((class1_tr, class2_tr))                 #Combine class1 training and class2 training data to get total training data
tr_targets = np.array([0 if i < len(tr_data) / 2 else 1 for i in range(len(tr_data))])           #Class labels. Assign the value "0" to class1 and the value "1" to class2.

# #Generate testing data.
class1_te = np.random.laplace(mean1, cov[0][0], (200,2))           #Class1 testing data
class2_te = np.random.laplace(mean2, cov[1][1], (200,2))            #Class2 testing data
te_data = np.concatenate((class1_te,class2_te))              #Combine class1 testing and class2 testing data to get total testing data
te_targets = np.array([0 if i < len(te_data) / 2 else 1 for i in range(len(te_data))])           #Class labels. Assign the value "0" to class1 and the value "1" to class2.

W, predictions = LS_Classify(tr_data,tr_targets,te_data)

print("Accuracy(Laplacian distribution ): ",LS_Classify_Accuracy(te_targets, predictions))

#visualizing decision boundary
class1_pred = []
class2_pred = []
for i, p in enumerate(predictions):
  if p != 1:
    class1_pred.append(te_data[i])
  else:
    class2_pred.append(te_data[i])

class1_pred = np.array(class1_pred)
class2_pred = np.array(class2_pred)
class1_te = np.array(class1_te)
class2_te = np.array(class2_te)

x = np.linspace(min(np.min(class1_te[:,0]), np.min(class2_te[:,0])) - 1, max(np.max(class1_te[:,0]), np.max(class2_te[:,0])) + 1, 1000)
y = decision_boundary(W, x)
fig, axes = plt.subplots(1, 2, figsize = (15,6))
fig.suptitle("Laplacian distribution")
axes[0].scatter(class1_te[:,0],class1_te[:, 1], label = "Class 1")
axes[0].scatter(class2_te[:,0],class2_te[:, 1], label = "Class 2")
axes[0].set_title("Original Classification of Points")
axes[0].grid(True)
axes[0].legend()

axes[1].scatter(class1_pred[:,0],class1_pred[:, 1], label = "Class 1 predicted")
axes[1].scatter(class2_pred[:,0],class2_pred[:, 1], label = "Class 2 predicted")
axes[1].plot(x, y, label = "Decision Boundary")
axes[1].set_title("Predicted Classification of Points")
axes[1].grid(True)
axes[1].legend()

plt.show()

#changing the targets
new_tr_targets = np.array([-1 if i < len(tr_data) / 2 else 1 for i in range(len(tr_data))])  # Assume -1 for class1 and +1 for class 2
new_te_targets = np.array([-1 if i < len(te_data) / 2 else 1 for i in range(len(te_data))])

W, predictions = LS_Classify(tr_data, new_tr_targets, te_data)
for i in range(len(predictions)):
  if predictions[i] == 0:
    predictions[i] = -1

print("New Accuracy: ",LS_Classify_Accuracy(new_te_targets, predictions))

class1_pred = []
class2_pred = []
for i, p in enumerate(predictions):
  if p != 1:
    class1_pred.append(te_data[i])
  else:
    class2_pred.append(te_data[i])

class1_pred = np.array(class1_pred)
class2_pred = np.array(class2_pred)
class1_te = np.array(class1_te)
class2_te = np.array(class2_te)

y = decision_boundary(W, x)
fig, axes = plt.subplots(1, 2, figsize = (15,6))
fig.suptitle("Plots with updated targets (Laplacian distribution)")
axes[0].scatter(class1_te[:,0],class1_te[:, 1], label = "Class 1")
axes[0].scatter(class2_te[:,0],class2_te[:, 1], label = "Class 2")
axes[0].set_title("Original Classification of Points")
axes[0].grid(True)
axes[0].legend()

axes[1].scatter(class1_pred[:,0],class1_pred[:, 1], label = "Class 1 predicted")
axes[1].scatter(class2_pred[:,0],class2_pred[:, 1], label = "Class 2 predicted")
axes[1].plot(x, y, label = "Decision Boundary")
axes[1].set_title("Predicted Classification of Points")
axes[1].grid(True)
axes[1].legend()

plt.show()


<b> Report your observations </b>

1.The decision boundary is linear in all the cases.

2.Because we are considering one-hat-K encoding of the targets, the choice of label does not affect the least square solution and the accuracy remains the same.

3.The accuracy for laplacian distribution is greater than that of gaussian distribution. It can be observed that least square solution is better for classfying in laplacian than for gaussian





<b> Part - (2) :  Linear & Fisher Discriminant analysis : </b> In this segment of the programming task, you'll learn the technique of projecting data from higher-dimensional to lower-dimensional space using both Linear Discriminant Analysis and Fisher Discriminant Analysis.

a). Data generation: Consider a classification scenario with two classes. Class 1 follows a Gaussian distribution with a mean vector $\begin{bmatrix} 1  \\ 1  \end{bmatrix}$ and a covariance matrix $\begin{bmatrix} 0.3 & 0.0  \\ 0.0 & 0.3 \end{bmatrix}$. Class 2 follows a Gaussian distribution with a mean vector $\begin{bmatrix} 2  \\ 2  \end{bmatrix}$ and a covariance matrix $\begin{bmatrix} 0.3 & 0.0  \\ 0.0 & 0.3 \end{bmatrix}$. Employ these class distributions to generate 50 samples per class for training and 200 samples for testing.

b). Projection using only the between-class covariance: Develop a function for performing projection from 2-D space to 1-D utilizing Linear Discriminant Analysis. It's important to emphasize that this method only considers the between-class covariance. Project the previously generated 2-D synthetic data into 1-D and conduct classification. Plot both the original and projected data points on the same graph.

c). Projection utilizing both between-class and within-class covariance: Create a function to perform projection from 2-D space to 1-D using Fisher Discriminant Analysis. It's essential to mention that this method considers both between-class covariance and within-class covariance. Project the previously generated 2-D synthetic data into 1-D and conduct classification. Plot both the original and projected data points on the same graph.

d). Extend Fisher discriminant analysis function to project the data from N dimenision to K dimension.

e). Report your observations.


In [ ]:
########################################
#Part-2)-(a): Data generation
########################################

#Set the parameters of class distribution
mean1 = [1,1]           #Mean of class1
mean2 = [2,2]           #Mean of class2
cov = [[0.3,0],[0,0.3]] #Covariance matrix. Same covariance for class1 and class2

#Generate training data. You can use "np.random.multivariate_normal()" function to sample data points from multivariate Gaussian distribution
class1_tr = np.random.multivariate_normal(mean1, cov, 50)           #Class1 training data
class2_tr = np.random.multivariate_normal(mean2, cov, 50)            #Class2 training data
tr_data = np.concatenate((class1_tr, class2_tr))              #Combine class1 training and class2 training data to get total training data
tr_targets = np.array([0 if i < len(tr_data) / 2 else 1 for i in range(len(tr_data))])          #Class labels. Assign the value "0" to class1 and the value "1" to class2.

#Generate testing data.
class1_te = np.random.multivariate_normal(mean1, cov, 200)            #Class1 testing data
class2_te = np.random.multivariate_normal(mean2, cov, 200)            #Class2 testing data
te_data = np.concatenate((class1_te, class2_te))              #Combine class1 testing and class2 testing data to get total testing data
te_targets = np.array([0 if i < len(te_data) / 2 else 1 for i in range(len(te_data))])           #Class labels. Assign the value "0" to class1 and the value "1" to class2.

########################################
#Part-2)-(b)
########################################
#Complete the below function
def LDA_classify(X_train, Y_train, X_test):
  #Inputs: Training data, Training labels, and Testing data
  #Outputs: Between class covariance

  #Compute the weight vector
  m1 = np.zeros_like(X_train[0])
  m2 = np.zeros_like(X_train[0])
  for i, x in enumerate(X_train):
    if i < len(X_train) / 2:
      m1 += x
    else:
      m2 += x

  m1 /= (len(X_train) / 2)
  m2 /= (len(X_train) / 2)

  w = (m2 - m1) / (np.linalg.norm(m2 - m1))
  #Compute the threshold
  threshold = (0.5) * (w.T @ (m1 + m2))

  #Project the data onto 1 dimensional space. Compare it with the threshold to make the decision
  projected_data = X_test @ w
  predictions = []
  for e in projected_data:
    if e > threshold:
      predictions.append(1)
    else:
      predictions.append(0)

  #Return weight vector and the predictions on testing data

  return predictions, w

predictions, w = LDA_classify(tr_data, tr_targets, te_data)

def line(w,x):
  return w[0] + w[1]*x -2

def LDA_project(w, X_test, Y_test):
  #Inputs: weight vector, testing data, and testing labels

  #Project the test data onto 1-dimensional space
  projected_data = X_test @ w
  class1_projected = []
  class2_projected = []

  for i in range(len(Y_test)):
    if Y_test[i] == 0:
      class1_projected.append(projected_data[i])
    else:
      class2_projected.append(projected_data[i])

  class1_projected = np.array(class1_projected)
  class2_projected = np.array(class2_projected)
  #Plot the projections
  y1 = line(w, class1_projected)
  y2 = line(w, class2_projected)
  plt.figure(figsize = (10,5))
  plt.scatter(X_test[: int(len(X_test) / 2), 0], X_test[:int(len(X_test) / 2), 1], label = "Class 1 Original Points")
  plt.scatter(X_test[int(len(X_test) / 2) : , 0], X_test[int(len(X_test) / 2) : , 1], label = "Class 2 Original Points")
  plt.scatter(class1_projected, y1, label = "Class1 Projected")
  plt.scatter(class2_projected, y2, label = "Class2 Projected")
  plt.title("Dimensionality reduction through LDA approach")
  plt.grid(True)
  plt.legend()
  plt.show()

LDA_project(w, te_data, te_targets)
#Compute and print the accuracy
count = 0
for i, p in enumerate(predictions):
  if p == te_targets[i]:
    count += 1

acc = count / len(predictions)
print('Accuracy with LDA:', acc*100)

########################################
#Part-2)-(c)
########################################
def FDA_classify(X_train, Y_train, X_test):
  #Inputs: Training data, Training labels, and Testing data
  #Outputs: Weight vector and predictions

  #Compute the weight vector
  m1 = np.zeros_like(X_train[0])
  m2 = np.zeros_like(X_train[0])
  for i, x in enumerate(X_train):
    if i < len(X_train) / 2:
      m1 += x
    else:
      m2 += x

  m1 /= (len(X_train) / 2)
  m2 /= (len(X_train) / 2)

  S1 = np.zeros((2,2))
  S2 = np.zeros((2,2))

  for i, x in enumerate(X_train):
    if i < len(X_train) / 2:
      S1 += np.outer(x - m1, x - m1)
    else:
      S2 += np.outer(x - m2, x - m2)

  S1 /= (len(X_train) / 2)
  S2 /= (len(X_train) / 2)

  S_w = S1 + S2

  v = np.linalg.inv(S_w) @ (m2 - m1)

  w = v / np.linalg.norm(v)

  #Compute the threshold
  threshold = (0.5) * (w.T @ (m1 + m2))

  #Project the data onto 1 dimensional space. Compare it with the threshold to make the decision
  projected_data = X_test @ w
  predictions = []
  for e in projected_data:
    if e > threshold:
      predictions.append(1)
    else:
      predictions.append(0)

  #Return weight vector and the predictions on testing data

  return w, predictions

w, predictions = FDA_classify(tr_data, tr_targets, te_data)

def FDA_project(w, X_test, Y_test):
  #Inputs: weight vector, testing data, and testing labels

  #Project the test data onto 1-dimensional space
  projected_data = X_test @ w
  class1_projected = []
  class2_projected = []

  for i in range(len(Y_test)):
    if Y_test[i] == 0:
      class1_projected.append(projected_data[i])
    else:
      class2_projected.append(projected_data[i])

  class1_projected = np.array(class1_projected)
  class2_projected = np.array(class2_projected)

  y1 = line(w, class1_projected)
  y2 = line(w, class2_projected)
  #Plot the projections
  plt.figure(figsize = (10,5))
  plt.scatter(X_test[: int(len(X_test) / 2), 0], X_test[:int(len(X_test) / 2), 1], label = "Class 1 Original Points")
  plt.scatter(X_test[int(len(X_test) / 2) : , 0], X_test[int(len(X_test) / 2) : , 1], label = "Class 2 Original Points")
  plt.scatter(class1_projected, y1, label = "class 1 projected")
  plt.scatter(class2_projected, y2, label = "class 2 projected")
  plt.title("Dimensionality reduction through FDA approach")
  plt.grid(True)
  plt.legend()
  plt.show()

FDA_project(w, te_data, te_targets)
#Compute and print the accuracy
count = 0
for i, p in enumerate(predictions):
  if p == te_targets[i]:
    count += 1

acc = count / len(predictions)

print('Accuracy with FDA:', acc*100)

# ########################################
# #Part-2)-(d)
# ########################################
def FDA_project_K(X_train, Y_train, X_test, k):
  #Inputs: Training data, Training label, Testing data, and the number of eigen vectors to be considered
  #Oupur: Projection matrix

  m1 = np.zeros_like(X_train[0])
  m2 = np.zeros_like(X_train[0])
  for i, x in enumerate(X_train):
    if i < len(X_train) / 2:
      m1 += x
    else:
      m2 += x

  m1 /= (len(X_train) / 2)
  m2 /= (len(X_train) / 2)





  #Compute and return the projection matrix
  return w



<b> Report your observations </b>

1.The projected points lie on the line $y = w^{T}x$ where $w$ is different in LDA and FDA approaches.

2.The accuracy of FDA is less than the accuracy of LDA

3.



<b> Part (3) : Classification using perceptron algorithm </b>

a). Data generation: Consider a classification scenario with two classes. Class 1 follows a Gaussian distribution with a mean vector $\begin{bmatrix} 1  \\ 1  \end{bmatrix}$ and a covariance matrix $\begin{bmatrix} 0.3 & 0.0  \\ 0.0 & 0.3 \end{bmatrix}$. Class 2 follows a Gaussian distribution with a mean vector $\begin{bmatrix} 2  \\ 2  \end{bmatrix}$ and a covariance matrix $\begin{bmatrix} 0.3 & 0.0  \\ 0.0 & 0.3 \end{bmatrix}$. Employ these class distributions to generate 50 samples per class for training and 200 samples for testing.


b). Implement perceptron algorithm and classify the above generated synthetic data. Plot the decision boundary/regions.

c). Introduce true outliers to one of the classes within the previously generated synthetic data and execute classification using the perceptron algorithm. Illustrate the decision boundary/regions. Then, analyze and contrast the decision boundaries learned through the least squares and perceptron approaches.

d). Let us consider four Gaussian distributions with mean vectors as $\begin{bmatrix} 0  \\ 0  \end{bmatrix}$, $\begin{bmatrix} 0  \\ 1  \end{bmatrix}$, $\begin{bmatrix} 1  \\ 1  \end{bmatrix}$, and $\begin{bmatrix} 1  \\ 0  \end{bmatrix}$, respectively. The covariance matrix is the same for all four Gaussian distributions, and the matrix is $\begin{bmatrix} 0.3 & 0.0  \\ 0.0 & 0.3 \end{bmatrix}$. Sample 60 data points from each distribution to get 240 data points. Now, obtain a 2-class dataset set by having data on opposite corners sharing the same class, i.e., data points sampled from Gaussian distributions with mean vectors $\begin{bmatrix} 0  \\ 0  \end{bmatrix}$ and $\begin{bmatrix} 1  \\ 1  \end{bmatrix}$ belong to class 1, and the data points sampled from the other two distributions belong to class 2. Assign class 1 data points with the label +1 and class 2 with the label -1. Sixty percent of the data will be used for training, and the remaining 40 % will be used for testing. You can see that it represents the XOR problem. Classify this data set using the perceptron algorithm.
  

e). Report your observations


In [ ]:
########################################
#Part-3)-(a): Data generation
########################################

#Set the parameters of class distribution
mean1 = [1,1]           #Mean of class1
mean2 = [2,2]           #Mean of class2
cov = [[0.3,0],[0,0.3]] #Covariance matrix. Same covariance for class1 and class2

#Generate training data. You can use "np.random.multivariate_normal()" function to sample data points from multivariate Gaussian distribution
class1_tr = np.random.multivariate_normal(mean1, cov, 50)           #Class1 training data
class2_tr = np.random.multivariate_normal(mean2, cov, 50)            #Class2 training data
tr_data = np.concatenate((class1_tr, class2_tr))              #Combine class1 training and class2 training data to get total training data
tr_targets = np.array([0 if i < len(tr_data) / 2 else 1 for i in range(len(tr_data))])          #Class labels. Assign the value "0" to class1 and the value "1" to class2.

#Generate testing data.
class1_te = np.random.multivariate_normal(mean1, cov, 200)            #Class1 testing data
class2_te = np.random.multivariate_normal(mean2, cov, 200)            #Class2 testing data
te_data = np.concatenate((class1_te, class2_te))              #Combine class1 testing and class2 testing data to get total testing data
te_targets = np.array([0 if i < len(te_data) / 2 else 1 for i in range(len(te_data))])           #Class labels. Assign the value "0" to class1 and the value "1" to class2.

########################################
#Part-3)-(b)
########################################
def Perceptron(X_train, Y_train, X_test, label = 0):
    #Inputs: Training data, Training labels, Testing data
    #Outputs: Testing data predictions, Weight vector representing the decision boundary

    epochs = 200   #Number of epochs
    X = []         #Data. Append ones to the training data to take care of the bias
    for x in X_train:
      X.append(np.array([1, x[0],x[1]]))

    X = np.array(X)
    w = np.array([2.0, -20.0, 10.0])    #Initialize the weight vector
    for epoch in range(epochs):
      #Compute the predictions
      predictions = X @ w

      #Compute the error
      for i,prediction in enumerate(predictions):
        if Y_train[i] == 1 and prediction < 0:
          w += X[i]
        elif Y_train[i] != 1 and prediction > 0:
          w -= X[i]

    #Compute the test data predicitions using the final weight vector
    Y = []
    for y in X_test:
      Y.append(np.array([1, y[0], y[1]]))

    Y = np.array(Y)
    targets = Y @ w
    predictions = []
    for e in targets:
      if e > 0:
        predictions.append(1)
      elif e < 0:
        if label != -1:
          predictions.append(0)
        else:
          predictions.append(-1)

    predictions = np.array(predictions)

    #Return the test data predictions and the final weight vector
    return predictions, w

##########################################
#Plot the decision boundary using perceptron
##########################################
def decision_boundary1(w, x):
  return (-w[1] / w[2])*x - (w[0] / w[2])

predictions, w = Perceptron(tr_data, tr_targets, te_data)
# X = []
# for x in tr_data:
#   X.append(np.array([1, x[0],x[1]]))

# X = np.array(X)

# targets = X @ w
# predictions1 = []
# for e in targets:
#   if e > 0:
#     predictions1.append(1)
#   elif e < 0:
#     predictions1.append(0)

# predictions1 = np.array(predictions1)
# print(predictions1)

#print(predictions)
class1_pred = []
class2_pred = []
for i, p in enumerate(predictions):
  if p != 1:
    class1_pred.append(te_data[i])
  else:
    class2_pred.append(te_data[i])

class1_pred = np.array(class1_pred)
class2_pred = np.array(class2_pred)

x = np.linspace(min(np.min(class1_te[:,0]), np.min(class2_te[:,0])) - 1, max(np.max(class1_te[:,0]), np.max(class2_te[:,0])) + 1, 1000)
y_1 = decision_boundary1(w, x)
plt.figure(figsize = (10,6))
plt.title("Decison Boundary using Perceptron")
plt.scatter(class1_pred[:,0],class1_pred[:, 1], label = "Class 1 predicted")
plt.scatter(class2_pred[:,0],class2_pred[:, 1], label = "Class 2 predicted")
plt.plot(x, y_1, label = "Decision Boundary")
plt.grid(True)
plt.legend()
plt.show()

##########################################
#Plot the decision boundary using least squares
##########################################
W, predicions = LS_Classify(tr_data, tr_targets, te_data)
y_ = decision_boundary(W,x)

class1_pred = []
class2_pred = []
for i, p in enumerate(predictions):
  if p != 1:
    class1_pred.append(te_data[i])
  else:
    class2_pred.append(te_data[i])

class1_pred = np.array(class1_pred)
class2_pred = np.array(class2_pred)

plt.figure(figsize = (10,6))
plt.title("Decison Boundary using Least Square")
plt.scatter(class1_pred[:,0],class1_pred[:, 1], label = "Class 1 predicted")
plt.scatter(class2_pred[:,0],class2_pred[:, 1], label = "Class 2 predicted")
plt.plot(x, y_, label = "Decision Boundary")
plt.grid(True)
plt.legend()
plt.show()

##########################################
#Part-3)-(c): Repeat the experiments with the true outliers added to the data. Compare and contrast the perceptron algorithm with least squares method
##########################################
outliers = np.random.uniform(-4, -5, (10,2,))   # 10 points as outliers for class 1
outliers_target = np.array([0 for i in range(len(outliers))])

tr_data = np.concatenate((tr_data, outliers))
tr_targets = np.concatenate((tr_targets,outliers_target))

predictions_perceptron, w_new = Perceptron(tr_data, tr_targets, te_data)
W_new, predictions_LS = LS_Classify(tr_data, tr_targets, te_data)
#print(tr_targets)
x = np.linspace(min(np.min(class1_te[:,0]), np.min(class2_te[:,0])) - 1, max(np.max(class1_te[:,0]), np.max(class2_te[:,0])) + 1, 1000)
y1 = decision_boundary1(w_new, x)
y2 = decision_boundary(W_new, x)

#print(w_new, "\n", w)
class1_pred = []
class2_pred = []
for i, p in enumerate(predictions_perceptron):
  if p != 1:
    class1_pred.append(te_data[i])
  else:
    class2_pred.append(te_data[i])

class1_pred = np.array(class1_pred)
class2_pred = np.array(class2_pred)

plt.figure(figsize = (10,6))
plt.title("Decison Boundary using Perceptron after adding outliers")
plt.scatter(class1_pred[:,0],class1_pred[:, 1], label = "Class 1 predicted")
#plt.scatter(tr_data[:, 0], tr_data[:, 1], label = "Training points")
plt.scatter(class2_pred[:,0],class2_pred[:, 1], label = "Class 2 predicted")
plt.plot(x, y1, color = "green",label = "Decision Boundary after adding outliers")
plt.plot(x, y_1, color = "black" ,label = "Decision Boundary before adding outliers")
plt.grid(True)
plt.legend()
plt.show()

class1_pred = []
class2_pred = []
for i, p in enumerate(predictions_LS):
  if p != 1:
    class1_pred.append(te_data[i])
  else:
    class2_pred.append(te_data[i])

class1_pred = np.array(class1_pred)
class2_pred = np.array(class2_pred)

#print(predictions_LS, "\n", W_new)

plt.figure(figsize = (10,6))
plt.title("Decison Boundary using Least Square after adding outliers")
plt.scatter(class1_pred[:,0],class1_pred[:, 1], label = "Class 1 predicted")
#plt.scatter(tr_data[:, 0], tr_data[:, 1], label = "Training points")
plt.scatter(class2_pred[:,0],class2_pred[:, 1], label = "Class 2 predicted")
plt.plot(x, y2, color = "green",label = "Decision Boundary after adding outliers")
plt.plot(x, y_ ,color = "black", label = "Decison Boundary before adding outliers")
plt.grid(True)
plt.legend()
plt.show()

##########################################
#Part-3)-(d): Data generation XOR
##########################################

#Set the parameters of Gaussian distribution
class1_mean1 = [0,0]     #Mean of Gaussian1
class1_mean2 = [4,4]     #Mean of Gaussian3
class2_mean1 = [4,0]     #Mean of Gaussian2
class2_mean2 = [0,4]     #Mean of Gaussian4

#Generate  data. You can use "np.random.multivariate_normal()" function to sample data points from multivariate Gaussian distribution

class1_data1 = np.random.multivariate_normal(class1_mean1, cov, 60)
class1_data2 = np.random.multivariate_normal(class1_mean2, cov, 60)
class2_data1 = np.random.multivariate_normal(class2_mean1, cov, 60)
class2_data2 = np.random.multivariate_normal(class2_mean2, cov, 60)

#Obtain 2-class dataset
class1_data = np.concatenate((class1_data1, class1_data2))
class2_data = np.concatenate((class2_data1, class2_data2))

dataset = np.concatenate((class1_data,class2_data))
targets = np.array([1 if i < (len(dataset) / 2) else -1 for i in range(len(dataset))])
#Create train test splits
tr_data = np.concatenate((dataset[:72],dataset[-72:]))
tr_targets = np.concatenate((targets[:72],targets[-72:]))

te_data = np.concatenate((dataset[72:120], dataset[-120:-72]))
te_targets = np.concatenate((targets[72:120], targets[-120:-72]))

#Classify this dataset using perceptron algorithm
predictions, w = Perceptron(tr_data, tr_targets, te_data, -1)

#Plot the decision boundary
class1_pred = []
class2_pred = []
for i, p in enumerate(predictions):
  if p != 1:
    class1_pred.append(te_data[i])
  else:
    class2_pred.append(te_data[i])

class1_pred = np.array(class1_pred)
class2_pred = np.array(class2_pred)

x = np.linspace(min(np.min(tr_data[:,0]), np.min(te_data[:,0])) - 1, max(np.max(tr_data[:,0]), np.max(te_data[:,0])) + 1, 1000)
y = decision_boundary1(w, x)
plt.figure(figsize = (10,6))
plt.title("Decison Boundary using Perceptron")
plt.scatter(class1_data[:,0],class1_data[:, 1], label = "Class 1")
plt.scatter(class2_data[:,0],class2_data[:, 1], label = "Class 2")
plt.plot(x, y, label = "Decision Boundary")
plt.grid(True)
plt.legend()
plt.show()

<b> Report your observations </b>

1.The decision boundary in case of perceptron algorithm is almost similar to that obtained from least square

2.After adding outliers, the decision boundary in case of perceptron alogrithm is not much affected but in case of least square, it gets tilted towards the outliers

3.In case of XOR problem, the perceptron algorithm is not able to classify them and the decision boundary keeps changing everytime we run the code.



<b> Part (4) : Understanding the decision boundaries of MAP approach to classification - </b> Decision boundary in 2 class classification problem is the locus of points satisfying $p(c_{1}/x)$ = $p(c_{2}/x)$. Where $c_{1}$ and $c_{2}$ are the class indicators and $x$ is the data. With proper assumpuptions on prior and likelihood, p(c/x) follows Gaussian distribution with appropriate parameters. Given the parameters of the Guassian distribution for the two classes i.e $\mu_{1},\Sigma_{1},\mu_{2},\Sigma_{2}$, we can derive the decision boundary equation i.e W and $w_{0}$. You may refer to class notes for the derivations and final equations. In this part of programming assignment we expect you to code the decision boundaries for the Gaussian distribution case and understand them.

**a). Class distributions share the same covariance matrix: Linear decision boundary.**
  0. Let $\mu_{1} = \begin{bmatrix} 1  \\ 1 \end{bmatrix}$, $\mu_{2} = \begin{bmatrix} 4  \\ 1 \end{bmatrix}$, and $\Sigma_{1}= \Sigma_{2} = \Sigma =  \begin{bmatrix} 0.1 & 0.0  \\ 0.0 & 0.1 \end{bmatrix}$
  1. Generate 100 samples from each class. This data set is used to understand the decision boundaries when $p(c_{1}) = p(c_{2})$
  2. Generate 50 samples from class1 and 150 samples from class2. This data set is used to understand the decision boundaries when $p(c_{1}) < p(c_{2})$
  3. Generate 150 samples from class1 and 50 samples from class2. This data set is used to understand the decision boundaries when $p(c_{1}) > p(c_{2})$
  4. Plot the decision boundaries for all the three cases in three different subplots.


**b). Class distributions have different covariance matrices: Non-linear decision boundary.**
  0. Let $\mu_{1} = \begin{bmatrix} 1  \\ 1 \end{bmatrix}$, $\mu_{2} = \begin{bmatrix} 3  \\ 1 \end{bmatrix}$, $\Sigma_{1}=   \begin{bmatrix} 0.2 & 0.0  \\ 0.0 & 0.02 \end{bmatrix}$, and $\Sigma_{2}=   \begin{bmatrix} 0.02 & 0.0  \\ 0.0 & 0.2 \end{bmatrix}$
  1. Generate 100 samples from each class. This data set is used to understand the decision boundaries when $p(c_{1}) = p(c_{2})$
  2. Generate 50 samples from class1 and 150 samples from class2. This data set is used to understand the decision boundaries when $p(c_{1}) < p(c_{2})$
  3. Generate 150 samples from class1 and 50 samples from class2. This data set is used to understand the decision boundaries when $p(c_{1}) > p(c_{2})$
  4. Plot the decision boundaries for all the three cases in three different subplots.


**c). Report your observations**




In [ ]:
##########################################
#Part-4a)-0: Data Generation
##########################################
m1 = [1,1]
m2 = [4,1]
cov = np.array([[0.1,0],[0,0.1]])

#Case1: Generate 100 samples from each class. You may use "np.random.multivariate_normal()" function to sample data points of multi-variate Gaussian distribution
class1_data_1 = np.random.multivariate_normal(m1, cov, 100)
class2_data_1 = np.random.multivariate_normal(m2, cov, 100)

#Case2: Generate 50 samples from class1 and 150 samples from class2. You may use "np.random.multivariate_normal()" function to sample data points of multi-variate Gaussian distribution
class1_data_2 = np.random.multivariate_normal(m1, cov, 50)
class2_data_2 = np.random.multivariate_normal(m2, cov, 150)

#Case3: Generate 150 samples from class1 and 50 samples from class2. You may use "np.random.multivariate_normal()" function to sample data points of multi-variate Gaussian distribution
class1_data_3 = np.random.multivariate_normal(m1, cov, 150)
class2_data_3 = np.random.multivariate_normal(m2, cov, 50)

##########################################
#Part-4a)-4: Plot the decision regions for all the three cases
##########################################
#Complete the below function
def decisionBoundary(X_train, Y_train):
    #Inputs: Training data and Training labels
    #Outputs: Decision boundary parameters, i.e., w0, w
    p_c1 = len(X_train[Y_train == 0]) / len(X_train)
    p_c2 = 1. - p_c1

    mean_1 = np.mean(X_train[Y_train == 0], axis = 0)
    mean_2 = np.mean(X_train[Y_train == 1], axis = 0)

    cov1 = np.zeros((2,2))
    cov2 = np.zeros((2,2))

    for i, x in enumerate(X_train):
      if i < len(X_train)*p_c1:
        cov1 += np.outer(x - mean_1, x - mean_1)
      else:
        cov2 += np.outer(x - mean_2, x - mean_2)

    cov1 /= (p_c1*len(X_train))
    cov2 /= (p_c2*len(X_train))

    cov = cov1 + cov2

    w = 2*(np.linalg.inv(cov) @ (mean_2 - mean_1))
    w0 = (mean_1.T @ np.linalg.inv(cov) @ mean_1) - (mean_2.T @ np.linalg.inv(cov) @ mean_2) - np.log(p_c1/p_c2)

    return np.array([w0]+list(w))

#Plot the decision regions
#Case1: p(c1) = p(c2)
tr_data1 = np.concatenate((class1_data_1,class2_data_1))
tr_targets1 = np.array([0 if i < len(tr_data1) / 2 else 1 for i in range(len(tr_data1))])

w = decisionBoundary(tr_data1, tr_targets1)

x = np.linspace(np.min(tr_data1[:,0]) - 1, np.max(tr_data1[:,0]) + 1, 1000)
y = decision_boundary1(w, x)
plt.figure(figsize = (10,6))
plt.title(r"Decison Boundary using MAP estimate where $p(c1) = p(c2)$")
plt.scatter(class1_data_1[:,0],class1_data_1[:, 1], label = "Class 1")
plt.scatter(class2_data_1[:,0],class2_data_1[:, 1], label = "Class 2")
plt.plot(x, y, label = "Decision Boundary")
plt.grid(True)
plt.legend()
plt.show()

#Case2: p(c1) < p(c2)
tr_data2 = np.concatenate((class1_data_2,class2_data_2))
tr_targets2 = np.array([0 if i < len(tr_data2) / 2 else 1 for i in range(len(tr_data2))])

w = decisionBoundary(tr_data2, tr_targets2)

x = np.linspace(np.min(tr_data2[:,0]) - 1, np.max(tr_data2[:,0]) + 1, 1000)
y = decision_boundary1(w, x)
plt.figure(figsize = (10,6))
plt.title(r"Decison Boundary using MAP estimate where $p(c1) < p(c2)$")
plt.scatter(class1_data_2[:,0],class1_data_2[:, 1], label = "Class 1")
plt.scatter(class2_data_2[:,0],class2_data_2[:, 1], label = "Class 2")
plt.plot(x, y, label = "Decision Boundary")
plt.grid(True)
plt.legend()
plt.show()

#Case3: p(c1) > p(c2)
tr_data3 = np.concatenate((class1_data_3,class2_data_3))
tr_targets3 = np.array([0 if i < len(tr_data3) / 2 else 1 for i in range(len(tr_data3))])

w = decisionBoundary(tr_data3, tr_targets3)

x = np.linspace(np.min(tr_data3[:,0]) - 1, np.max(tr_data3[:,0]) + 1, 1000)
y = decision_boundary1(w, x)
plt.figure(figsize = (10,6))
plt.title(r"Decison Boundary using MAP estimate where $p(c1) > p(c2)$")
plt.scatter(class1_data_3[:,0],class1_data_3[:, 1], label = "Class 1")
plt.scatter(class2_data_3[:,0],class2_data_3[:, 1], label = "Class 2")
plt.plot(x, y, label = "Decision Boundary")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
##########################################
#Part-4b)-0: Data Generation
##########################################
m1 = np.array([1,1])
m2 = np.array([3,1])
cov1 = np.array([[0.2, 0.0],[0.0, 0.02]])
cov2 = np.array([[0.02, 0.0],[0.0, 0.2]])

#Case1: Generate 100 samples from each class. You may use "np.random.multivariate_normal()" function to sample data points of multi-variate Gaussian distribution
class1_data_1 = np.random.multivariate_normal(m1, cov1, 100)
class2_data_1 = np.random.multivariate_normal(m2, cov2, 100)

#Case2: Generate 50 samples from class1 and 150 samples from class2. You may use "np.random.multivariate_normal()" function to sample data points of multi-variate Gaussian distribution
class1_data_2 = np.random.multivariate_normal(m1, cov1, 50)
class2_data_2 = np.random.multivariate_normal(m2, cov2, 150)

#Case3: Generate 150 samples from class1 and 50 samples from class2. You may use "np.random.multivariate_normal()" function to sample data points of multi-variate Gaussian distribution
class1_data_3 = np.random.multivariate_normal(m1, cov1, 150)
class2_data_3 = np.random.multivariate_normal(m2, cov2, 50)

##########################################
#Part-4b)-4: Plot the decision regions for all the three cases
##########################################
#Complete the below function
def decisionBoundary(X_train, Y_train, X_test):
    #Inputs: Training data and Training labels
    #Outputs: Decision boundary parameters, i.e., w0, w
    p_c1 = len(X_train[Y_train == 0]) / len(X_train)
    p_c2 = 1. - p_c1

    mean_1 = np.mean(X_train[Y_train == 0], axis = 0)
    mean_2 = np.mean(X_train[Y_train == 1], axis = 0)

    cov1 = np.zeros((2,2))
    cov2 = np.zeros((2,2))

    for i, x in enumerate(X_train):
      if i < len(X_train)*p_c1:
        cov1 += np.outer(x - mean_1, x - mean_1)
      else:
        cov2 += np.outer(x - mean_2, x - mean_2)

    cov1 /= (p_c1*len(X_train))
    cov2 /= (p_c2*len(X_train))

    W = np.linalg.inv(cov1) - np.linalg.inv(cov2)
    w = 2*(np.linalg.inv(cov2) @ mean_2 - np.linalg.inv(cov1) @ mean_1)
    w0 = (mean_1.T @ np.linalg.inv(cov1) @ mean_1) - (mean_2.T @ np.linalg.inv(cov2) @ mean_2) - np.log(p_c1/p_c2) + 0.5*np.log(np.linalg.det(cov1) / np.linalg.det(cov2))

    return W, np.array([w0]+list(w))

#Plot the decision regions
def decision_boundary2(W,w,x,y):
  return W[0][0]*(x**2) + W[1][1]*(y**2) + 2*W[0][1]*x*y + w[1]*x + w[2]*y + w[0]

#Case1: p(c1) = p(c2)
tr_data1 = np.concatenate((class1_data_1,class2_data_1))
tr_targets1 = np.array([0 if i < len(tr_data1) / 2 else 1 for i in range(len(tr_data1))])

W, w = decisionBoundary(tr_data1, tr_targets1, [])

x_min, x_max = tr_data1[:, 0].min() - 1, tr_data1[:, 0].max() + 1
y_min, y_max = tr_data1[:, 1].min() - 1, tr_data1[:, 1].max() + 1

xx,yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

Z = decision_boundary2(W,w,xx,yy)
plt.figure(figsize = (10,6))
plt.title(r"Decision Boundary when $p(c1) = p(c2)$")
plt.scatter(class1_data_1[:,0],class1_data_1[:, 1], label = "Class 1")
plt.scatter(class2_data_1[:,0],class2_data_1[:, 1], label = "Class 2")
plt.contour(xx, yy, Z, levels = [0], linewidths = 2, colors = 'black')
plt.grid(True)
plt.legend()
plt.show()

#Case2: p(c1) < p(c2)
tr_data2 = np.concatenate((class1_data_2,class2_data_2))
tr_targets2 = np.array([0 if i < len(tr_data2) / 2 else 1 for i in range(len(tr_data2))])

W, w = decisionBoundary(tr_data2, tr_targets2, [])

x_min, x_max = tr_data2[:, 0].min() - 1, tr_data2[:, 0].max() + 1
y_min, y_max = tr_data2[:, 1].min() - 1, tr_data2[:, 1].max() + 1

xx,yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

Z = decision_boundary2(W,w,xx,yy)
plt.figure(figsize = (10,6))
plt.title(r"Decision Boundary when $p(c1) < p(c2)$")
plt.scatter(class1_data_2[:,0],class1_data_2[:, 1], label = "Class 1")
plt.scatter(class2_data_2[:,0],class2_data_2[:, 1], label = "Class 2")
plt.contour(xx, yy, Z, levels = [0], linewidths = 2, colors = 'black')
plt.grid(True)
plt.legend()
plt.show()

#case3: p(c1) > p(c2)
tr_data3 = np.concatenate((class1_data_3,class2_data_3))
tr_targets3 = np.array([0 if i < len(tr_data3) / 2 else 1 for i in range(len(tr_data3))])

W, w = decisionBoundary(tr_data3, tr_targets3, [])

x_min, x_max = tr_data3[:, 0].min() - 1, tr_data3[:, 0].max() + 1
y_min, y_max = tr_data3[:, 1].min() - 1, tr_data3[:, 1].max() + 1

xx,yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

Z = decision_boundary2(W,w,xx,yy)
plt.figure(figsize = (10,6))
plt.title(r"Decision Boundary when $p(c1) > p(c2)$")
plt.scatter(class1_data_3[:,0],class1_data_3[:, 1], label = "Class 1")
plt.scatter(class2_data_3[:,0],class2_data_3[:, 1], label = "Class 2")
plt.contour(xx, yy, Z, levels = [0], linewidths = 2, colors = 'black')
plt.grid(True)
plt.legend()
plt.show()



<b> Report your observations </b>

1.In case of shared covariance, the decision boundary is linear. While in case where covariances are not equal, the decision boundary is non-linear.

2.In case of shared covariacne, the line shifts towards the class whose prior pobability is greater. In case where the probabilities are equal, the line is in between them

3.The same is in the case of unequal covariances. The decision boundary centers around the class whose prior probabilty is greater



<b> Part-5) Classification using logistic regression ( Iterative reweighted least squares approach ): </b> logistic regression model: $y = \frac{1}{1+e^{-\textbf{w}^{T}\textbf{x}}}$. With the provided training data, X_train and Y_train, your task is to identify the optimal $\textbf{w}$ that accurately predicts y based on the input $\textbf{x}$. Subsequently, this $\textbf{w}$ can be applied to predict outcomes on the test data, X_test.





a). Data generation: Consider a classification scenario with two classes. Class 1 follows a Gaussian distribution with a mean vector $\begin{bmatrix} 1  \\ 1  \end{bmatrix}$ and a covariance matrix $\begin{bmatrix} 0.3 & 0.0  \\ 0.0 & 0.3 \end{bmatrix}$. Class 2 follows a Gaussian distribution with a mean vector $\begin{bmatrix} 2  \\ 2  \end{bmatrix}$ and a covariance matrix $\begin{bmatrix} 0.3 & 0.0  \\ 0.0 & 0.3 \end{bmatrix}$. Employ these class distributions to generate 50 samples per class for training and 200 samples for testing.

b). Create a function called "Logistic_Regression" that accepts training data, training labels, and testing data as inputs. The function should aim to discover the optimal $\textbf{w}$ using the training data. You can initialize $\textbf{w}$ with random values and iteratively update it to determine the optimal $\textbf{w}$. Subsequently, this optimal $\textbf{w}$ can be employed to make predictions on the test data.

c). Evaluate the classification performance, i.e., compute the accuracy on the test data.

d). Write a function to generate & visualize the decision regions, either by showing the boundary line or by using different colors for the two regions. Overlay the test points using scatter.

e). Introduce genuine outliers to one of the class datasets generated previously. Then, conduct Least Squares classification and Logistic Regression on this modified dataset. Plot the decision boundaries/regions and provide insights into the differences between their outcomes.


f). Report your observations


In [ ]:
##################################################
#Part-5)-1: Data generationgenerationgenerationgeneration
##################################################
import numpy as np
import matplotlib.pyplot as plt

#Set tthehe parameters of class distributiondistribution
mean1 = [1,1]           #Mean of class1
mean2 = [2,2]           #Mean of class2
cov = [[0.3,0],[0,0.3]] #Covariance matrix. Same covariance for class1 and class2

#Generate training data. You can use "np.random.multivariate_normal()" function to sample data points from multivariate Gaussian distribution
class1_tr = np.random.multivariate_normal(mean1, cov, 50)         #Class1 training data
class2_tr = np.random.multivariate_normal(mean2, cov, 50)            #Class2 training data
tr_data = np.concatenate((class1_tr, class2_tr))              #Combine class1 training and class2 training data to get total training data
tr_targets = np.array([0 if i < len(tr_data) / 2 else 1 for i in range(len(tr_data))])          #Class labels. Assign the value "0" to class1 and the value "1" to class2.

#Generate testing data.
class1_te = np.random.multivariate_normal(mean1, cov, 200)            #Class1 testing data
class2_te = np.random.multivariate_normal(mean2, cov, 200)            #Class2 testing data
te_data = np.concatenate((class1_te, class2_te))              #Combine class1 testing and class2 testing data to get total testing data
te_targets = np.array([0 if i < len(te_data) / 2 else 1 for i in range(len(te_data))])        #Class labels. Assign the value "0" to class1 and the value "1" to class2.


def sigmoid(a):
  return 1 / (1 + np.exp(-a))
##################################################
#Part-5)-b: Complete the below logistic regression function
##################################################
def Logistic_Regression(training_data, training_label, testing_data):

  tolerance = 0.01
  # Initialize weights with random values
  weights = np.random.randn(training_data.shape[1], 1)
  y = training_data @ weights
  y = 1 / (1 + np.exp(-y))
  k = 0
  g = training_data.T @ (y - training_label)
  while (np.linalg.norm(g) > tolerance and k < 1000):
    y = training_data @ weights
    y = 1 / (1 + np.exp(-y - 1e-5))
    R = np.zeros((len(y),len(y)))
    i = 0
    j = 0
    while((i < len(y)) and (j < len(y))):
      R[i,j] = y[i] * (1 - y[i])
      i += 1
      j += 1

    h = training_data.T @ R @ training_data
    g = training_data.T @ (y - training_label)
    I = np.eye(h.shape[0])
    delta = np.linalg.inv(h + 1e-5*I) @ g
    weights = weights - delta
    k += 1

  return weights

ones = np.ones((tr_data.shape[0], 1))
tr_data1 = np.concatenate((ones, tr_data), axis=1)
ones = np.ones((te_data.shape[0], 1))
te_data1 = np.concatenate((ones, te_data), axis=1)
tr_targets1 = tr_targets.reshape((len(tr_targets),1))
weights = Logistic_Regression(tr_data1, tr_targets1, te_data1)
tr_targets = tr_targets.reshape(len(tr_targets))

###############################################
#Part-5)-c: Accuracy
###############################################
def LR_Accuracy(Y_test, Y_pred):
  #Inputs: Ground truth and predicted lables of test data
  #Outputs: Accuracy
  Y_pred = [0 if prediction < 0 else 1 for prediction in Y_pred]
  return np.mean(Y_pred == Y_test)

Y_pred = te_data1 @ weights
acc = LR_Accuracy(te_targets, Y_pred)
print('\n\n Accuracy with Logistic Regression:', acc, '\n\n')

##########################################
#Part-5)-d: Plot the decision regions. Overlay the test points on the plot
##########################################
def plot_decision_regions(training_data, training_label,weights):

    x = np.linspace(-1, 5, 100)
    def boundary(w,x):
      return (-w[1] / w[2] )*x - (w[0] / w[2])

    y = boundary(weights,x)
    class1_data = training_data[training_label == 0]
    class2_data = training_data[training_label == 1]
    plt.figure(figsize=(10, 6))
    plt.scatter(class1_data[:, 0], class1_data[:, 1], color='blue', label='Class 1')
    plt.scatter(class2_data[:, 0], class2_data[:, 1], color='red', label='Class 2')
    plt.plot(x,y,color = 'black', label = 'Decision Boundary')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title('Decision Regions')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_decision_regions(tr_data, tr_targets,weights)
plot_decision_regions(te_data, te_targets,weights)

<b> Part (6) : Estimating the parameters of Gaussian Mixture Models (GMM) using Expectation-Maximization (EM) algorithm </b>


<dt> <b> a. Generating synthetic data : </b>  This part describes the steps involved in generating the synthetic data. </dt>


<dt>  1. Let us consider the GMM with Bivariate Gaussians   </dt>
<dd>  - Choose the appropriate means ($\mu$), covariances ($\Sigma$), and weights ($\pi$) of Bivariate Gaussian distributions. You may choose the following parameters for this asignment.  </dd>
<br>
<dt>  <center> $ \pi = \begin{bmatrix} 0.6  \\ 0.4 \end{bmatrix} \;\;\;\; \mu_{1} = \begin{bmatrix} -1.0  \\ -1.0 \end{bmatrix} \;\;\;\; \mu_{2} = \begin{bmatrix} 1.0  \\ 1.0 \end{bmatrix} \;\;\;\; $ </center>   </dd>
<br>
<dt>  <center> $ \Sigma_{1}^{full} = \begin{bmatrix} 0.15 & 0.1  \\ 0.1 & 0.25 \end{bmatrix} \;\;\;\; \Sigma_{2}^{full} = \begin{bmatrix} 0.3 & -0.25  \\ -0.25 & 0.25 \end{bmatrix}  $ </center>  </dd>
<br>
<dt>  <center> $ \Sigma_{1}^{diagonal} = \begin{bmatrix} 0.1 & 0.0  \\ 0.0 & 0.2 \end{bmatrix} \;\;\;\; \Sigma_{2}^{diagonal} = \begin{bmatrix} 0.2 & 0.0  \\ 0.0 & 0.1 \end{bmatrix}  $ </center>  </dd>
<br>
<dt>  <center> $ \Sigma_{1}^{spherical} = \begin{bmatrix} 0.2 & 0.0  \\ 0.0 & 0.2 \end{bmatrix} \;\;\;\; \Sigma_{2}^{spherical} = \begin{bmatrix} 0.1 & 0.0  \\ 0.0 & 0.1 \end{bmatrix}  $ </center>  </dd>

<dd>  - Pick one Gaussian following the selection probability as  $\pi$    </dd>
<dd>  - Sample the data point from the selected Bivariate Gaussian distribution    </dd>
<dd>  - Repeat the process N times to get the entire data set  </dd>
<dt>  2.Create the synthetic data set with the number of mixtures as 2.  </dt>
<dt>  3.Plot the synthetic data set with unique colors to the data points drawn from each Bivariate Gaussian distribution in GMM  </h4> </dt> </dd>

<dt> <b> b. Implementation of Expectation and Maximization (EM) algorithm : </b> This part focuses on implementing the EM algorithm from scratch to estimate the parameters of the Gaussian Mixture Model (GMM). </dt>  

<dt>  1. Chose the number of mixtures (M) and initialize the parameters of GMM model  </dt>
<dd>  - Make sure that the diagonal elements of covariance are positive, and the weights of Gaussians are positive and sum to 1 for satisfying the axioms of probability.  </dd>

<dt>   2. Expectation step :  Evaluate the responsibility of Gaussians in generating the data points using the current estimated parameters i.e $\hat\mu_{k}$ , $\hat\Sigma_{k}$ and $\hat\pi_{k}$, here k indicates the Gaussian in GMM  </dt>
<br>

<dd>  <center> $\gamma_{nk} = \frac{\pi_{k} N ( \textbf{x}_{n} / \mu_{k} , \Sigma_{k} )}{ \sum_{m=1}^{M} \pi_{m} N ( x_{n} / \mu_{m} , \Sigma_{m} ) }    \;\;\;\;\;\; ∀ \;\;k=1,2,...,M \;\; and \;\; n=1,2,...,N$  </center>  </dd>

<br>

<dd>   Where N is the total number of data points and M is the total number of Gaussians in GMM   </dd>

<dd>   - $\gamma_{nk}$ represents the responsibility of $k^{th}$ Gaussian in generating the $n^{th}$ data point   </dd>

<dd>   - $\sum_{n=1}^{N}$ $\gamma_{nk}$ represents the the effective number of data points generated by $k^{th}$ Gaussian   </dd>


<dt>   3. Maximization step :  Update the parameters ($\hat\mu_{k}$ , $\hat\Sigma_{k}$ and $\hat\pi_{k}$) of GMM using the current resosibilities ($\gamma_{nk}$) by maximizing the likelihood. </dt>

<dd>
 - Effective number of data points generated by $k^{th}$ Gaussian : $N_{k}$ = $\sum_{n=1}^{N}$ $\gamma_{nk}$   
 </dd>

<dd>
 - Mean of $k^{th}$ Gaussian : $\hat{\mu_{k}} = \frac{1}{N_{k}} \sum_{n=1}^{N}\gamma_{nk} x_{n}$   
 </dd>

<dd>
 - Covariance of $k^{th}$ Gaussian : $\hat{\Sigma_{k}} = \frac{1}{N_{k}} \sum_{n=1}^{N} \gamma_{nk} ( x_{n} - \hat{\mu_{k}} )( x_{n} - \hat{\mu_{k}} )^{T}$   
 </dd>

<dd>
 - Weight of $k^{th}$ Gaussian : $\hat{\pi_{k}} = \frac{N_{k}}{N}$  
 </dd>

<dt>  4. Compute the log-likelihood with the updated parameters  </dt>
<dt>   5. Repeat the expectation and maximization steps untill convergence  </dt>

<dt>  <b> c. Illustration of EM iterations </b>  </dt>

<dt>   5. Plot the contours of the estimated distributions over the iterations in the EM algorithm and overlay the data points.  </dt>

<dt>  <b> d. Understanding the issues with EM algorithm </b>  </dt>
<dt>   6. Check the influence of initialization on the convergence of the EM algorithm and fit quality.  </dt>
<dd>   - Demonstrate the observations with the plots mentioned in the "Illustration of EM iterations" section    </dd>
<dt>   6. Check the influence of the type of covariance matrix on the convergence of the EM algorithm and fit quality . </dt>
<dd>   - Execute the EM algorithm with different covariance matrices i.e Spherical, Diagonal and Full covariance  </dd>
<dd>   - Demonstrate the observations with the plots mentioned in the "Illustration of EM iterations" section  </dd>


In [ ]:
#All imports
import numpy as np
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from scipy.stats import norm, multivariate_normal
import sys
import math

##########################################
#Part-6)-a: Generating synthetic data
##########################################

def generateData(pi, mu, sigma, N):
    ############################################################################
    #pi = weights of Gaussians, mu = mean of Gaussians, sigma = convariance of Gaussians, N = number of data points
    #Steps
    #1.Select th Gaussian following the probability distribution as pi
    #2.Sample the data points from the selected Gaussian
    #3.Repeat the process N times to get N data points
    #4.Return the sampled data
    ############################################################################
    #You may look into "random.choices" and "np.random.multivariate_normal" functions
    data = []
    labels = []
    for _ in range(N):
      choice = np.random.choice([0,1], p = pi)
      point = np.random.multivariate_normal(mu[choice], sigma[choice])
      data.append(point)
      labels.append(choice)

    return np.asarray(data), np.asarray(labels)
    ############################################################################


############################################################################
#Chose the appropriate means ($\mu$), covariances ($\Sigma$), and weights
############################################################################
mu1 = np.array([-1.0,-1.0])   #Mean vector of component1 in GMM
sig1 = np.array([[0.15,0.1],[0.1,0.25]])  #Full covariance matrix of component1 in GMM
mu2 = np.array([1.0,1.0])   #Mean vector of component2 in GMM
sig2 = np.array([[0.3,-0.25],[-0.25,0.25]])  #Full covariance matrix of component2 in GMM
pi = np.array([0.6,0.4])    #Prior probabilities

n_points = 200 #Number of points

# Spherical covariances
sig1_s = np.array([[0.2,0],[0,0.2]])
sig2_s = np.array([[0.1,0],[0,0.1]])
# Diagonal covariances
sig1_d = np.array([[0.1,0],[0,0.2]])
sig2_d = np.array([[0.2,0],[0,0.1]])

############################################################################
#Create the synthetic data set
############################################################################

data_full, labels_full = generateData(pi, [mu1, mu2], [sig1, sig2], 200)
data_sp, labels_sp = generateData(pi, [mu1, mu2], [sig1_s, sig2_s], 200)
data_dg, labels_dg = generateData(pi, [mu1, mu2], [sig1_d, sig2_d], 200)

############################################################################
#Plot the synthetic data set
############################################################################

fig, ax = plt.subplots(1,3, figsize=(20,7))

#Visualize the data sampled from the GMM with full covariance matrix
ax[0].set_title('Full Covariance')
ax[0].scatter(data_full[:,0], data_full[:, 1], color = 'red')
ax[0].grid(True)

#Visualize the data sampled from the GMM with spherical covariance matrix
ax[1].set_title('Spherical Covariance')
ax[1].scatter(data_sp[:,0], data_sp[:, 1], color = 'blue')
ax[1].grid(True)
#Visualize the data sampled from the GMM with diagonal covariance matrix
ax[2].set_title('Diagonal Covariance')
ax[2].scatter(data_dg[:,0], data_dg[:,1], color = 'green')
ax[2].grid(True)

plt.show()

In [ ]:
#All imports
import numpy as np
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
import sys
import math

##########################################
#Part-6)-b: Implimentation of Expectation-Maximization (EM) algorithm
##########################################
def total_probability(point, pi, mu, sigma):
  sum = 0
  for k in range(len(pi)):
    sum += pi[k] * multivariate_normal.pdf(point, mean = mu[k], cov = sigma[k])
  return sum

############################################################################
#Write a function to compute the log-likelihood of the data given the parameters
############################################################################
def likelihood(data, pi, mu, sigma, log_likelihood=False, likelihoods=False):
  #Inputs: data, parameters of GMM model
  #Outputs: likelihood
  n = len(data)
  likelihood = 0
  for point in data:
    likelihood += np.log(total_probability(point, pi, mu, sigma))

  sum = 0
  for prob in pi:
    sum += prob

  likelihood -= n*(sum - 1)

  if log_likelihood:
    return likelihood
  else:
    return np.exp(likelihood)




############################################################################
#Write a function for the expectation step
############################################################################
def expectation(data, pi, mu, sigma):
  #Inputs: data, parameters of GMM model
  #Outputs: gamma
  gamma = np.zeros((len(data), len(pi)))
  for n in range(len(data)):
    #Iterate through the data points
    for k in range(len(pi)):
      #Iterate through the components in GMM
      numerator = pi[k] * multivariate_normal.pdf(data[n], mean = mu[k], cov = sigma[k])
      denominator = total_probability(data[n], pi, mu,sigma)
      gamma[n,k] = numerator/denominator
  return gamma



############################################################################
#Write a function for the maximization step.
############################################################################
def maximization(data, gamma):
  #Inputs: data, gamma
  #Outputs: Updated parameters of GMM model
  N = len(data)
  M = len(gamma[0])
  pi = np.zeros(M)
  mu = np.zeros((M,2))
  sigma = np.zeros((M,2,2))

  #Update pi
  N_k = np.zeros(M)
  for k in range(M):
    for n in range(N):
      N_k[k] += gamma[n,k]
    pi[k] = N_k[k]/N

  #Update mu
  for k in range(M):
    for n in range(N):
      mu[k] += gamma[n,k] * data[n]

    mu[k] /= N_k[k]

  #Update sigma
  for k in range(M):
    for n in range(N):
      x = data[n] - mu[k]
      sigma[k] += (gamma[n,k] * np.outer(x,x))

    sigma[k] /= N_k[k]

  #Return the parameters of GMM model
  return pi, mu, sigma


#Complete the below function. It plots the contours
def plot_contours(data, labels, pi, mu, sigma):
  x = np.linspace(np.min(data[:,0]) - 1, np.max(data[:,0] + 1), 100)
  y = np.linspace(np.min(data[:,1]) - 1, np.max(data[:,1] + 1), 100)
  X, Y = np.meshgrid(x, y)
  J = np.zeros((len(x), len(y)))
  for i in range(len(x)):
    for j in range(len(y)):
      J[i,j] = likelihood(np.array([x[i], y[j]]), pi, mu, sigma, log_likelihood=True)

  plt.figure(figsize = (10,6))
  plt.title("Contour Plot")
  plt.contour(X, Y, J, levels = np.linspace(np.min(J), np.max(J), 100))
  plt.scatter(data[:,0], data[:,1], c = labels)
  plt.xlabel("Feature 1")
  plt.ylabel("Feature 2")
  plt.grid(True)
  plt.show()

############################################################################
#Run the expectation and maximzation algorithm to estimate the parameters
############################################################################

# Diagonal Covariance
data = data_dg   #data to run EM-algorithm
labels = labels_dg  #labels
N = len(data)       #number of input examples
D = data.shape[1]       #data dimension
M = 2       #no.of components in GMM

x = np.random.rand()
pi = np.array([x, 1- x])    #Initialize the prior probabilities
mu = np.random.rand(*(M,2))      #Initialize the mean vectors
sigma1 = np.array([[np.random.uniform(1,2),0],[0,np.random.rand()]])
sigma2 = np.array([[np.random.uniform(1,2), 0],[0, np.random.rand()]])
sigma = np.array([sigma1, sigma2])   #Initialize the covariance matrices


nsteps = 18  #Number of steps to run EM algorithm
for index in range(0,nsteps,1):

  #Compute the likelihood
  ll = likelihood(data, pi, mu, sigma, log_likelihood=True)

  #Plot the contours
  plot_contours(data, labels, pi, mu, sigma)

  #Expectation step
  gamma = expectation(data, pi, mu, sigma)

  #Maximization step
  pi, mu, sigma = maximization(data, gamma)


print(pi, '\n', mu , '\n', sigma)

In [ ]:
############################################################################
##Part-6)-d: Understand the issues with EM algorithm
############################################################################
#Spherical covariance
data = data_sp   #data to run EM-algorithm
labels = labels_sp  #labels
N = len(data)       #number of input examples
D = data.shape[1]       #data dimension
M = 2       #no.of components in GMM

x = np.random.rand()
pi = np.array([x, 1- x])    #Initialize the prior probabilities
mu = np.random.rand(*(M,2))      #Initialize the mean vectors
sigma1 = np.array([[np.random.uniform(1,2),0],[0,np.random.rand()]])
sigma2 = np.array([[np.random.uniform(1,2), 0],[0, np.random.rand()]])
sigma = np.array([sigma1, sigma2])   #Initialize the covariance matrices


nsteps = 18  #Number of steps to run EM algorithm
for index in range(0,nsteps,1):

  #Compute the likelihood
  ll = likelihood(data, pi, mu, sigma, log_likelihood=True)

  #Plot the contours
  plot_contours(data, labels, pi, mu, sigma)

  #Expectation step
  gamma = expectation(data, pi, mu, sigma)

  #Maximization step
  pi, mu, sigma = maximization(data, gamma)


print(pi, '\n', mu , '\n', sigma)


In [ ]:
# Full Covariance matrix
data = data_dg   #data to run EM-algorithm
labels = labels_dg  #labels
N = len(data)       #number of input examples
D = data.shape[1]       #data dimension
M = 2       #no.of components in GMM

x = np.random.rand()
pi = np.array([x, 1- x])    #Initialize the prior probabilities
mu = np.random.rand(*(M,2))      #Initialize the mean vectors
x = np.random.rand()
sigma1 = np.array([[np.random.uniform(1,2),x],[x,np.random.rand()]])
sigma2 = np.array([[np.random.uniform(1,2), x],[x, np.random.rand()]])
sigma = np.array([sigma1, sigma2])   #Initialize the covariance matrices


nsteps = 18  #Number of steps to run EM algorithm
for index in range(0,nsteps,1):

  #Plot the contours
  plot_contours(data, labels, pi, mu, sigma)

  #Expectation step
  gamma = expectation(data, pi, mu, sigma)

  #Maximization step
  pi, mu, sigma = maximization(data, gamma)


print(pi, '\n', mu , '\n', sigma)